In [1]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np

import random, copy, time

In [2]:
start_time = time.time()

In [3]:
data = pd.read_csv('../datasets/MK01.csv')
data

,Job,Operação,M1,M2,M3,M4,M5,M6
0,1,1,5,0,4,0,0,0
1,1,2,0,1,5,0,3,0
2,1,3,0,0,4,0,0,2
3,1,4,1,6,0,0,0,5
4,1,5,0,0,1,0,0,0
5,1,6,0,0,6,3,0,6
6,2,1,0,6,0,0,0,0
7,2,2,0,0,1,0,0,0
8,2,3,2,0,0,0,0,0
9,2,4,0,6,0,6,0,0


In [4]:
class Job:
    
    def __init__(self, numero, tempos):
        self.n = numero
        self.operacao_a_processar = 0
        self.fim_op_anterior = 0
        self.tempos_processamento = tempos # [[tempos maq1], [tempos maq2], [tempos maq3]]

    def processar_prox_operacao(self, maquina):
        #print('operacao' + str(self.operacao_a_processar) + 'do job ' + str(self.n))
        tempo_processamento = self.tempos_processamento[self.operacao_a_processar][maquina]
        if (self.operacao_a_processar < len(self.tempos_processamento)):
            self.operacao_a_processar += 1
        return tempo_processamento


    def __repr__(self):
        return 'Job ' + str(self.n)

In [5]:
class Maquina():
    
    def __init__(self, num):
        self.num = num
        #self.operacoes = list()
        self.tempo_de_processamento = 0

    def __repr__(self):
        return 'Máquina ' + str(self.num) #+ ' Operações escalonadas = ' + str(self.operacoes)

In [6]:
class Cromossomo:

    def __init__(self, operacoes, maquinas):
        self.operacoes = operacoes
        self.maquinas = maquinas

    def __repr__(self):
        return 'Operacoes ' + str(self.operacoes) + ', Máquinas ' + str(self.maquinas)

In [7]:
n_jobs = data['Job'].unique()
jobs = []
for i in range(1, len(n_jobs) + 1):
    data_arr = np.asanyarray(data)
    subdata = []
    for _ in range(len(data_arr)):
        if data_arr[_][0] == i:
            subdata.append(data_arr[_][2:])
            
    job = Job(i, subdata)
    jobs.append(job)
jobs

[Job 1, Job 2, Job 3, Job 4, Job 5, Job 6, Job 7, Job 8, Job 9, Job 10]

In [8]:
n_maquinas = data.shape[1] - 2
maquinas = []
for _ in range(n_maquinas):
    maquina = Maquina(_)
    maquinas.append(maquina)
maquinas

[Máquina 0, Máquina 1, Máquina 2, Máquina 3, Máquina 4, Máquina 5]

In [9]:
def selecionar_maquina_randomica(job):
    while True:
        idx = random.randrange(0, len(job.tempos_processamento[job.operacao_a_processar]))
        if (job.tempos_processamento[job.operacao_a_processar - 1][idx] != 0):
            return idx

In [10]:
def get_cromossomo(jobs):

    jobs_escalonar = copy.deepcopy(np.asarray(data['Job']))
    random.shuffle(jobs_escalonar)
    maquinas_sorteadas = []
    # Sorteando job a ser escalonado
    for idx_job in jobs_escalonar:
        job = jobs[idx_job - 1]
        #print(job)
        maquina_sorteada = selecionar_maquina_randomica(job)
        maquinas_sorteadas.append(maquina_sorteada)

    maquinas_sorteadas = np.asarray(maquinas_sorteadas)

    return Cromossomo(jobs_escalonar, maquinas_sorteadas)

In [11]:
def populacao_inicial(n):
    populacao = []
    for _ in range(n):
        populacao.append(get_cromossomo(jobs))
        
    return populacao

In [12]:
def fitness(jobs, maquinas, cromossomo):
    for _ in range(len(cromossomo.operacoes)):
        op = cromossomo.operacoes[_]
        n_maquina = cromossomo.maquinas[_]
        job = jobs[op - 1]
        #print('processando ' + repr(job) + 'na maquina ' + str(n_maquina))
        tempo_operacao = job.processar_prox_operacao(n_maquina)
        if tempo_operacao == 0:
            while tempo_operacao == 0:
                tempo_operacao = job.tempos_processamento[job.operacao_a_processar - 1][random.randint(0, n_maquinas - 1)]
                if tempo_operacao != 0:
                    break
            #print(cromossomo)
        #print(tempo_operacao)
        maquina = maquinas[n_maquina]
        if (job.operacao_a_processar > 0):
            if job.fim_op_anterior > maquina.tempo_de_processamento:
                maquina.tempo_de_processamento = job.fim_op_anterior + tempo_operacao
            else:
                maquina.tempo_de_processamento += tempo_operacao
                
            job.fim_op_anterior = maquina.tempo_de_processamento
    
    

    makespan = 0
    for maquina in maquinas:
        if maquina.tempo_de_processamento > makespan:
            makespan = maquina.tempo_de_processamento

    return makespan

In [13]:
def selecao(pop, porcentagem):
    result = []
    for _ in range(round(len(pop) * porcentagem)):
        result.append(random.choice(pop))
    return result

In [14]:
def selecionar_jobs(unique_jobs):
    lista = list(unique_jobs)
    jobs_pai = set(random.choices(lista, k=random.randint(1, len(unique_jobs))))
    jobs_mae = unique_jobs - jobs_pai
    jobs_pai = list(jobs_pai)
    jobs_mae = list(jobs_mae)

    return jobs_pai, jobs_mae

def buscar_operacao(posicao, job, cromossomo):
    operacoes = cromossomo.operacoes
    maquinas = cromossomo.maquinas
    print(operacoes[posicao])
    print(job)
    if operacoes[posicao] != job:
        #print('operacoes[posicao] != job, pesquisando prox instancia')
        for i in range(posicao + 1, len(operacoes)):
            if operacoes[i] == job:
                #print('Encontrei, retornando', operacoes[i])
                return operacoes[i], maquinas[i]

    return operacoes[posicao], maquinas[posicao]


def cruzamento(pop):
    selecao = pop
    unique_jobs = set(pop[0].operacoes)
    while len(pop) < tamanho_pop:
        idx_pai = random.randint(0,len(selecao) - 2)
        idx_mae = random.randint(idx_pai,len(selecao) - 1)
        pai = selecao[idx_pai]
        mae = selecao[idx_mae]
        jobs_pai, jobs_mae = selecionar_jobs(unique_jobs)
        #print('jobs pesquisados no pai =', jobs_pai)
        #print('jobs pesquisados na mae =', jobs_mae)
        novas_ops = np.zeros(len(pai.operacoes), dtype=int)
        novas_maqs = np.zeros(len(pai.maquinas), dtype=int)
        jobs_a_buscar = copy.deepcopy(pai.operacoes).tolist()

        for pos in range(len(pai.operacoes)):
            if pai.operacoes[pos] in jobs_pai:
                novas_ops[pos] = pai.operacoes[pos]
                novas_maqs[pos] = pai.maquinas[pos]

        for pos in range(len(mae.operacoes)):
            if mae.operacoes[pos] in jobs_mae:
                #print('copiando valor da mae', mae.operacoes[pos])
                    for i in range(len(novas_ops)):
                        if novas_ops[i] == 0:
                            novas_ops[i] = mae.operacoes[pos]
                            novas_maqs[i] = mae.maquinas[pos]
                            break
            
        novo_cromossomo = Cromossomo(np.asarray(novas_ops), np.asarray(novas_maqs))
        #print('apos cruzamento, novo cromossomo =', novo_cromossomo)
        pop.append(novo_cromossomo)

    return pop

In [15]:
def mutacao(cromossomo):
    index = random.randint(0, len(cromossomo.operacoes) - 1)
    operacoes = cromossomo.operacoes
    maquinas_crom = cromossomo.maquinas
    anterior = 0
    proximo = len(operacoes) - 1
    if index != 0:
        for i in range(index - 1, -1, -1):
            if operacoes[i] == operacoes[index]:
                anterior = i
                break

    if index != len(operacoes) - 1:
        for i in range(index + 1, len(operacoes)):
            if operacoes[i] == operacoes[index]:
                    proximo = i
                    break

    if (proximo - anterior > 1):
        #print('realizando mutacao, cromossomo inicial =', operacoes)
        index2 = random.randint(anterior + 1, proximo - 1)

        salvo = operacoes[index]
        operacoes[index] = operacoes[index2]
        operacoes[index2] = salvo

        job1 = jobs[operacoes[index] - 1]
        maquinas_crom[index] = selecionar_maquina_randomica(job1)
        job2 = jobs[operacoes[index2] - 1]
        maquinas_crom[index2] = selecionar_maquina_randomica(job2)
        #print('mutacao realizada, cromossomo final =', operacoes)
    
    return Cromossomo(operacoes, maquinas_crom)


In [16]:
def genetico(n_geracoes, prob_mutacao, tamanho_pop):
    pop_inicial = populacao_inicial(tamanho_pop)
    pop_atual = pop_inicial
    for _ in range(n_geracoes):
        next_gen = selecao(pop_atual, 0.5)
        next_gen = cruzamento(next_gen)
        for i in range(len(next_gen)):
            number = random.random()
            if number < prob_mutacao:
                cromo = next_gen[i]
                mutado = mutacao(cromo)
                next_gen[i] = mutado

        pop_atual = next_gen
        #print(pop_atual)

    return pop_atual

In [17]:
# Rodando o algoritmo 20 vezes para obter resultados médios de tempo de execução
resultados_makespans = []
resultados_tempos = []
for i in range(20):
    start_time = time.time()
    tamanho_pop = 100
    res_final = genetico(500, 0.3, tamanho_pop)
    makespans = []
    for cromo in res_final:
        fit = fitness(jobs, maquinas, cromo)
        makespans.append(fit)
        for job in jobs:
            job.fim_op_anterior = 0
            job.operacao_a_processar = 0

        for maquina in maquinas:
            maquina.tempo_de_processamento = 0
    makespans = np.asarray(makespans)
    #print('makespans', makespans)
    resultados_makespans.append(np.min(makespans))
    resultados_tempos.append(time.time() - start_time)
print(resultados_makespans)
print(resultados_tempos)
    

[53, 56, 57, 50, 47, 52, 51, 55, 55, 55, 54, 56, 56, 49, 53, 51, 58, 62, 55, 56]
[7.562210321426392, 8.13344693183899, 8.38736629486084, 7.754809617996216, 7.751607179641724, 7.789453029632568, 7.7288007736206055, 7.5554423332214355, 7.533689975738525, 7.584735155105591, 8.261632204055786, 7.5162694454193115, 7.6547465324401855, 7.541700124740601, 7.814556121826172, 7.86849570274353, 7.786670684814453, 8.194976806640625, 7.698081970214844, 7.458913564682007]


In [18]:
def calcular_novo_Q(alfa, gamma, tabela, linha, coluna):
    if linha < len(tabela) - 1:
        return (1 - alfa) * tabela[linha][coluna] + alfa*(1 + gamma * np.max(tabela[linha + 1]))
    else:
        return (1 - alfa) * tabela[linha][coluna] + alfa*(1 + gamma * np.max(tabela[linha]))

In [19]:
def genetico_treino(n_geracoes, prob_mutacao, pop_inicial):
    pop_atual = pop_inicial
    for _ in range(n_geracoes):
        next_gen = selecao(pop_atual, 0.5)
        next_gen = cruzamento(next_gen)
        for i in range(len(next_gen)):
            number = random.random()
            if number < prob_mutacao:
                cromo = next_gen[i]
                mutado = mutacao(cromo)
                next_gen[i] = mutado

        pop_atual = next_gen
        #print(pop_atual)

    return pop_atual
    
def treinar_tabela(tabela):
    for i in range(10):
        pop_inicial = populacao_inicial(tamanho_pop)
        acoes = [0.1, 0.2, 0.3, 0.4, 0.5]
        for i in range(5):
            res_makespan = []
            # print('executando geracoes ate ', (i + 1) * 100)
            pops_res = []
            for acao in acoes:
                res = genetico_treino(100, acao, pop_inicial)
                pops_res.append(res)
                makespans = []
                for cromo in res:
                    fit = fitness(jobs, maquinas, cromo)
                    makespans.append(fit)
                    for job in jobs:
                        job.fim_op_anterior = 0
                        job.operacao_a_processar = 0

                    for maquina in maquinas:
                        maquina.tempo_de_processamento = 0
                makespans = np.asarray(makespans)
                min_mkspn = np.min(makespans)
                res_makespan.append(min_mkspn)
            # print(res_makespan)
            res_makespan = np.asarray(res_makespan)
            min_idx = np.argmin(res_makespan)
            tabela[i][min_idx] = calcular_novo_Q(0.1, 0.8, tabela, i, min_idx)

            pop_inicial = pops_res[0]
    return tabela

In [20]:
tabela_Q = np.zeros((5,5))
print('tabela_inicial', tabela_Q)
tabela_Q = treinar_tabela(tabela_Q)
print('tabela_treinada', tabela_Q)

tabela_inicial [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
tabela_treinada [[0.21480768 0.24356479 0.29580768 0.21538368 0.13175169]
 [0.220096   0.         0.13329561 0.39689618 0.31764578]
 [0.41619508 0.         0.1164672  0.39732575 0.10864   ]
 [0.23478941 0.12357338 0.         0.30887546 0.40151981]
 [0.23478941 0.         0.23478941 0.22077338 0.38877386]]


In [21]:
def buscar_mutacao(idx_linha, tabela):
    linha = tabela[idx_linha]
    indice = np.argmax(linha)
    return (indice + 1) * 0.1

def genetico_hibrido(tamanho_pop, tabela_Q):
    pop_inicial = populacao_inicial(tamanho_pop)
    pop_atual = pop_inicial
    for _ in range(500):
        if _ % 100 == 0:
            linha = round(_ / 100)
            prob_mutacao = buscar_mutacao(linha, tabela_Q)
        next_gen = selecao(pop_atual, 0.5)
        next_gen = cruzamento(next_gen)
        for i in range(len(next_gen)):
            number = random.random()
            if number < prob_mutacao:
                cromo = next_gen[i]
                mutado = mutacao(cromo)
                next_gen[i] = mutado

        pop_atual = next_gen
        #print(pop_atual)

    return pop_atual

In [22]:
# Rodando o algoritmo 20 vezes para obter resultados médios de tempo de execução
resultados_makespans = []
resultados_tempos = []
for i in range(20):
    start_time = time.time()
    tamanho_pop = 100
    res_final = genetico_hibrido(tamanho_pop, tabela_Q)
    makespans = []
    for cromo in res_final:
        fit = fitness(jobs, maquinas, cromo)
        makespans.append(fit)
        for job in jobs:
            job.fim_op_anterior = 0
            job.operacao_a_processar = 0

        for maquina in maquinas:
            maquina.tempo_de_processamento = 0
    makespans = np.asarray(makespans)
    #print('makespans', makespans)
    resultados_makespans.append(np.min(makespans))
    resultados_tempos.append(time.time() - start_time)
print(resultados_makespans)
print(resultados_tempos)

[49, 49, 54, 58, 53, 54, 56, 52, 53, 58, 55, 51, 50, 56, 51, 53, 48, 58, 53, 54]
[5.450098991394043, 5.333094120025635, 5.473144054412842, 5.446187496185303, 5.532623529434204, 5.32387900352478, 5.299926519393921, 5.186691761016846, 5.422746419906616, 5.368892431259155, 5.165744304656982, 5.29903769493103, 5.4889936447143555, 5.249965190887451, 5.172637462615967, 5.460140705108643, 5.250078439712524, 5.008054256439209, 5.360300064086914, 5.3565309047698975]
